### Scrape výberových konaní

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import pickle
import sys
import time

In [2]:
# nacitaj relativne linky ku vsetkym konaniam
with open('./Data/rel_linky_list_20191118.pkl', 'rb') as f:
    relativne_linky = pickle.load(f)

# vytiahni indexy z relativnych linkov
indexy_konani = []
for link in relativne_linky:
    indexy_konani.append(link.split('/')[-1])

In [3]:
# definuj url
url_detail = 'https://open.slovensko.sk/VK/SelectionProcedureDetail/'
url_frame = 'https://open.slovensko.sk/VK/GetSelectionProcedureDetailFrame/'
url0 = 'https://open.slovensko.sk/VK/SelectionProcedureListSearch'

In [ ]:
datestring = '20191118'

In [4]:
# data ktore ulozime z SelectionProcedureDetailFrame
vars_dict = {} # label:var
vars_dict['Identifikátor výberového konania'] = 'vk_id'
vars_dict['Služobný úrad'] = 'sluzobny_urad'
vars_dict['Sídlo služobného úradu'] = 'sluzobny_urad_sidlo'
vars_dict['Druh výberoveho konania'] = 'vk_druh'
vars_dict['Počet obsadzovaných miest'] = 'pocet_obsadzovane'
vars_dict['Organizačný útvar'] = 'org_utvar'
vars_dict['Nadriadený organizačný útvar'] = 'nad_org_utvar'
vars_dict['Odbor štátnej služby'] = 'statna_sluzba_odbor'
vars_dict['Obsadzované štátnozamestnanecké miesto vo funkcii'] = 'funkcia'
vars_dict['Druh štátnej služby'] = 'statna_sluzba_druh'
vars_dict['Pravidelné miesto výkonu štátnej služby'] = 'miesto_vykonu'
vars_dict['Vzdelanie'] = 'vzdelanie'
vars_dict['Dátum vyhlásenia výberového konania'] = 'datum_vyhlasenia'
vars_dict['Termín na podanie žiadosti o zaradenie do výberového konania spolu s ďalšími požadovanými dokumentmi'] = 'datum_podania'
#
vars_dict['Vedúci štátny zamestnanec'] = 'veduci'
vars_dict['Najnáročnejšia činnosť'] = 'najnarocnejsia_cinnost'
vars_dict['Bližšie určená najnáročnejšia činnosť'] = 'najnarocnejsia_cinnost_blizsie'
vars_dict['Vyžaduje sa ovládanie cudzieho jazyka'] = 'cudzi_jazyk'
vars_dict['Vyžaduje sa odborná prax'] = 'odborna_prax'
vars_dict['Vyžaduje sa oprávnenie na oboznamovanie sa s utajovanými skutočnosťami'] = 'utajovane_skutocnosti'
vars_dict['Ďalšie požiadavky'] = 'dalsie_poziadavky'

# data, ktore vytiahneme z SelectionProcedureDetail
stlpce = ['index_konania', 'konanie_uspesne', 'pocet_vybrani', 'pocet_dalsi', 'pocet_neuspesni', 'pocet_prihlaseni']
# vsetky stahovane data
stlpce = stlpce + list(vars_dict.values())

# pre niektore upravme datove typy
stlpce_int = ['index_konania', 'pocet_vybrani', 'pocet_dalsi', 'pocet_neuspesni', 'pocet_prihlaseni', 'pocet_obsadzovane']
stlpce_bool = ['konanie_uspesne'] 
stlpce_datum = ['datum_vyhlasenia', 'datum_podania']
stlpce_string = list(filter(lambda x: x not in stlpce_int+stlpce_bool+stlpce_datum, stlpce))

In [27]:
# vytvorme dataframe
df = pd.DataFrame(columns=stlpce)

# nastavme dtype stlpcov
df[stlpce_int] = df[stlpce_int].astype(int)
df[stlpce_bool] = df[stlpce_bool].astype(bool)
df[stlpce_datum] = df[stlpce_datum].astype('datetime64[ns]')
df[stlpce_string] = df[stlpce_string].astype(str)

In [ ]:
# nepritomne k 20191118
# df_preskocene = pd.DataFrame(columns=['vysledok'])

In [28]:
driver = webdriver.Firefox()
driver.get(url0)

In [29]:
driver_reset_counter = 0
# driver_reset_counter = 16901
for index_konania in indexy_konani:   
    # resetuj driver raz za cas
    # dirty riesenie na zapratavanie pamate
    if driver_reset_counter != 0 and driver_reset_counter % 100 == 0:
        # print cas
        print(driver_reset_counter//100, time.localtime()[3], ':', time.localtime()[4])
        # zavri prehliadac
        driver.close()
        time.sleep(2)
        # uloz data
        with open('./Data/data_'+datestring+'_'+str(driver_reset_counter//100)+'.pkl', 'wb') as f:
            pickle.dump(df, f)
    
        #reset data frame
        df = pd.DataFrame(columns=stlpce)

        df[stlpce_int] = df[stlpce_int].astype(int)
        df[stlpce_string] = df[stlpce_string].astype(str)

        #reset driver
        driver = webdriver.Firefox()
        driver.get(url0)
        time.sleep(1)
    
    # nacitaj stranku s poctami a vysledkom
    url = url_detail + index_konania
    driver.get(url)
    time.sleep(0.1)
    soup = BeautifulSoup(driver.page_source)
    
    #
    # vysledok konania
    #

    vysledok = None
    for elem in soup.findAll("h2"):
        if 'výsledok' in elem.getText():
            vysledok = elem.getText()
        #elif 'Vyhlásené' in elem.getText():
        #    vysledok = -1

    if vysledok is None:
        # log VK ine ako tie s vyhlasenym vysledkom
        print('Preskocene i', index_konania, 'Text:', elem.getText())
        with open('log_preskocene_'+datestring+'.txt', 'a') as f:
            log_str = 'Preskocene ' + str(index_konania) + ' Text: ' + elem.getText() + '\n'
            f.write(log_str)
        continue # neukladaj
    else:
        if 'neúspešné' in vysledok:
            konanie_uspesne = False
        else:
            konanie_uspesne = True
            
    #
    # pocty uchadzacov
    #
    
    pocet_vybrani = 0
    pocet_dalsi = 0
    pocet_neuspesni = 0
    pocet_prihlaseni = 0
    for elem in soup.findAll("div", class_="subpagePnl"):
        popisok = elem.h4.getText()
        if 'Vybraní' in popisok or 'Úspešní uchádzači' in popisok:
            pocet_vybrani = len(elem.findAll('strong'))
        elif 'Ďalší' in popisok:
            pocet_dalsi = len(elem.findAll('strong'))
        elif 'Neúspešní' in popisok:
            pocet_neuspesni = len(elem.findAll('strong'))
        elif 'Počet' in popisok:
            pocet_prihlaseni = int(elem.p.getText().replace(' ', '').replace('\n', ''))
        else:
            raise Exception(popisok)
            
    # uloz data ziskane doteraz
    konanie_dict = {}
    konanie_dict['index_konania'] = index_konania
    konanie_dict['konanie_uspesne'] = konanie_uspesne
    konanie_dict['pocet_vybrani'] = pocet_vybrani
    konanie_dict['pocet_dalsi'] = pocet_dalsi
    konanie_dict['pocet_neuspesni'] = pocet_neuspesni
    konanie_dict['pocet_prihlaseni'] = pocet_prihlaseni
    
    # nacitaj stranku s informaciami SelectionProcedureDetailFrame
    url = url_frame + index_konania
    driver.get(url)
    time.sleep(0.1)
    soup = BeautifulSoup(driver.page_source)
    
    # nacitaj popisky
    # musi byt trochu vseobecne, nie vsetky polia su zhodne medzi konaniami
    labels = list(map(lambda x: str(x.getText()), soup.findAll('span', class_='fieldLabel')))
    # najdi datum
    datum_index = labels.index('Dátum vyhlásenia výberového konania')
    # nacitaj hodnoty
    vals = soup.findAll('div', class_="contentVis wordwrap")
    # pred datumom sa nachadzaju pozadovane dokumenty a forma podania ziadosti - nezaujimave a nezapada do struktury
    # vyhodime
    vals.pop(datum_index)
    vals.pop(datum_index)
    # linebreaky nahradim newline, nahradime nezlomitelnu medzeru \xa0 obycajnou medzerou
    vals = list(map(lambda x: str(x.getText("\n")).replace('\xa0', ' '), vals))
    
    # najdi a uloz data
    for field_label in vars_dict.keys():
        premenna = vars_dict[field_label]
        try:
            hodnota = vals[labels.index(field_label)]
        except ValueError:
            hodnota = ''

        if premenna in stlpce_string:
            konanie_dict[premenna] = hodnota
        if premenna in stlpce_int:
            if hodnota == '':
                hodnota = -1
            konanie_dict[premenna] = int(hodnota)
        elif premenna in stlpce_datum:
            datum_list = hodnota.split('.')
            datum_list = list(map(int, datum_list))
            konanie_dict[premenna] = pd.Timestamp(day=datum_list[0], month=datum_list[1], year=datum_list[2])

    df = df.append(konanie_dict, ignore_index=True)
    driver_reset_counter += 1

Preskocene i 19024 Text: Ukončené prihlasovanie
Preskocene i 19028 Text: Zrušené výberové konanie
Preskocene i 19045 Text: Zrušené výberové konanie
Preskocene i 19081 Text: Ukončené prihlasovanie
Preskocene i 19090 Text: Ukončené prihlasovanie
Preskocene i 19106 Text: Zrušené výberové konanie
Preskocene i 19112 Text: Zrušené výberové konanie
170 15 : 6
Preskocene i 19135 Text: Ukončené prihlasovanie
Preskocene i 19143 Text: Zrušené výberové konanie
Preskocene i 19158 Text: Zrušené výberové konanie
Preskocene i 19182 Text: Zrušené výberové konanie
Preskocene i 19206 Text: Zrušené výberové konanie
Preskocene i 19213 Text: Zrušené výberové konanie
171 15 : 8
Preskocene i 19265 Text: Ukončené prihlasovanie
Preskocene i 19266 Text: Ukončené prihlasovanie
Preskocene i 19269 Text: Ukončené prihlasovanie
Preskocene i 19279 Text: Zrušené výberové konanie
172 15 : 9
Preskocene i 19344 Text: Ukončené prihlasovanie
Preskocene i 19347 Text: Zrušené výberové konanie
Preskocene i 19379 Text: Zrušené 

Preskocene i 21611 Text: Ukončené prihlasovanie
Preskocene i 21613 Text: Ukončené prihlasovanie
Preskocene i 21617 Text: Ukončené prihlasovanie
Preskocene i 21620 Text: Ukončené prihlasovanie
Preskocene i 21622 Text: Ukončené prihlasovanie
183 15 : 28
Preskocene i 21637 Text: Ukončené prihlasovanie
Preskocene i 21643 Text: Ukončené prihlasovanie
Preskocene i 21648 Text: Ukončené prihlasovanie
Preskocene i 21650 Text: Ukončené prihlasovanie
Preskocene i 21653 Text: Ukončené prihlasovanie
Preskocene i 21654 Text: Ukončené prihlasovanie
Preskocene i 21657 Text: Ukončené prihlasovanie
Preskocene i 21659 Text: Ukončené prihlasovanie
Preskocene i 21660 Text: Ukončené prihlasovanie
Preskocene i 21651 Text: Ukončené prihlasovanie
Preskocene i 21661 Text: Ukončené prihlasovanie
Preskocene i 21662 Text: Ukončené prihlasovanie
Preskocene i 21658 Text: Ukončené prihlasovanie
Preskocene i 21663 Text: Ukončené prihlasovanie
Preskocene i 21664 Text: Ukončené prihlasovanie
Preskocene i 21667 Text: Uko

Preskocene i 21904 Text: Ukončené prihlasovanie
Preskocene i 21912 Text: Ukončené prihlasovanie
Preskocene i 21917 Text: Ukončené prihlasovanie
Preskocene i 21911 Text: Ukončené prihlasovanie
Preskocene i 21914 Text: Vyhlásené
Preskocene i 21915 Text: Ukončené prihlasovanie
184 15 : 32
Preskocene i 21919 Text: Ukončené prihlasovanie
184 15 : 32
Preskocene i 21920 Text: Ukončené prihlasovanie
184 15 : 33
Preskocene i 21871 Text: Ukončené prihlasovanie
184 15 : 33
Preskocene i 21684 Text: Ukončené prihlasovanie
184 15 : 33
Preskocene i 21858 Text: Ukončené prihlasovanie
184 15 : 33
Preskocene i 21913 Text: Ukončené prihlasovanie
Preskocene i 21895 Text: Vyhlásené
Preskocene i 21896 Text: Vyhlásené
Preskocene i 21898 Text: Vyhlásené
Preskocene i 21899 Text: Vyhlásené
Preskocene i 21921 Text: Ukončené prihlasovanie
Preskocene i 21923 Text: Ukončené prihlasovanie
Preskocene i 21922 Text: Ukončené prihlasovanie
Preskocene i 21925 Text: Ukončené prihlasovanie
Preskocene i 21926 Text: Ukončené

Preskocene i 22102 Text: Ukončené prihlasovanie
Preskocene i 22111 Text: Ukončené prihlasovanie
Preskocene i 22135 Text: Ukončené prihlasovanie
Preskocene i 22136 Text: Ukončené prihlasovanie
Preskocene i 22139 Text: Ukončené prihlasovanie
Preskocene i 22142 Text: Ukončené prihlasovanie
Preskocene i 22144 Text: Ukončené prihlasovanie
Preskocene i 22148 Text: Ukončené prihlasovanie
Preskocene i 22151 Text: Ukončené prihlasovanie
Preskocene i 22143 Text: Ukončené prihlasovanie
Preskocene i 22153 Text: Ukončené prihlasovanie
Preskocene i 22154 Text: Ukončené prihlasovanie
Preskocene i 22146 Text: Ukončené prihlasovanie
Preskocene i 22157 Text: Ukončené prihlasovanie
Preskocene i 22138 Text: Ukončené prihlasovanie
Preskocene i 22141 Text: Ukončené prihlasovanie
Preskocene i 22140 Text: Ukončené prihlasovanie
Preskocene i 22109 Text: Ukončené prihlasovanie
Preskocene i 22152 Text: Ukončené prihlasovanie
Preskocene i 22158 Text: Ukončené prihlasovanie
Preskocene i 22160 Text: Ukončené prihla

Preskocene i 22348 Text: Vyhlásené
Preskocene i 22349 Text: Vyhlásené
Preskocene i 22342 Text: Vyhlásené
Preskocene i 22350 Text: Vyhlásené
Preskocene i 22352 Text: Vyhlásené
Preskocene i 22353 Text: Vyhlásené
Preskocene i 22355 Text: Vyhlásené
Preskocene i 22351 Text: Vyhlásené
Preskocene i 22290 Text: Vyhlásené
Preskocene i 22358 Text: Vyhlásené
Preskocene i 22359 Text: Vyhlásené
Preskocene i 22360 Text: Vyhlásené
Preskocene i 22361 Text: Vyhlásené
Preskocene i 22110 Text: Vyhlásené
Preskocene i 22362 Text: Vyhlásené
Preskocene i 22363 Text: Vyhlásené
Preskocene i 22356 Text: Vyhlásené
Preskocene i 22364 Text: Vyhlásené
Preskocene i 22346 Text: Vyhlásené
Preskocene i 22366 Text: Vyhlásené
Preskocene i 22367 Text: Vyhlásené
Preskocene i 22369 Text: Vyhlásené
Preskocene i 22354 Text: Vyhlásené
Preskocene i 22375 Text: Vyhlásené
Preskocene i 22374 Text: Vyhlásené
Preskocene i 22371 Text: Vyhlásené
Preskocene i 22373 Text: Vyhlásené
Preskocene i 22365 Text: Vyhlásené
Preskocene i 22368 T

In [30]:
# uloz posledny df
with open('./Data/data_'+datestring+'_'+str(driver_reset_counter//100 + 1)+'.pkl', 'wb') as f:
            pickle.dump(df, f)

In [22]:
driver_reset_counter

16998